# Fluent Python
https://github.com/fluentpython/example-code

파이썬 용어집 

https://docs.python.org/ko/3/glossary.html

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import time
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated# Fluent Python

## CHAPTER 12 Inheritance: For Good or For Worse

- The pitfalls of subclassing from built-in types
- Multiple inheritance and the method resolution order

### Subclassing Built-In Types Is Tricky



In [3]:
class DoppelDict(dict):
    def __setitem__(self, key, value): # 저장할때 값을 복제
        super().__setitem__(key, [value] * 2)

In [4]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [5]:
dd['two'] = 2 
dd

{'one': 1, 'two': [2, 2]}

In [7]:
dd.update(three=3) # __setitem__ 을 호출하지 않음으로 three 의 값은 복제 되지 않음
dd

{'one': 1, 'two': [2, 2], 'three': 3}

내장 자료형은 '슈퍼클래스에서 구현된 메서드 안에서 호출하더라도 메서드 검색은 대상 객체(self)의 클래스에서 시작해야 한다' 는 객체지향 프로그래밍의 기본 규칙을 어기고 있다.

In [1]:
class AnswerDict(dict):
    def __getitem__(self, key): #
        return 42

In [2]:
ad = AnswerDict(a='foo')
ad['a']

42

In [5]:
d = {}
d.update(ad) # d 는 평범한 dict 객체며, 여기서는 ad 객체를 이용해서 갱신
d['a'] # dict.update() 메서드는 오버라이드된 AnswerDict.__getitem__() 무시
d

'foo'

{'a': 'foo'}

C 언어로 구형된 내장 자료형의 메서드를 위험하면 위와 같은 2가지 문제 발생 <br>
내장 자료형을 상속한 사용자 정의 클래스에 영향을 미침 <br>
파이썬으로 구현된 클래스를 상속하면 문제가 발생하지 않음 <br>

### Multiple Inheritance and Method Resolution Order


In [6]:
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
   
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [8]:
d = D()
C.pong(d)

PONG: <__main__.D object at 0x7f9ed10c64a8>


객체를 인수로 전달해서 슈퍼클래스의 메서드를 직접 호출 
파이썬이 상속 그래프를 조회할때는 특정한 순서를 따름 

메서드 결정 순서 Method Resolution Order(MRO) 

In [9]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

MRO 를 우회해서 슈퍼클래스 메서드 직접 호출 

In [10]:
def ping(self):
    A.ping(self) # instead of super().ping()
    print('post-ping:', self)

self 를 반드시 명식해야함

MRO 는 상속 그래프를 고려할 뿐 아니라 서브클래스 정의에 나열된 슈퍼클래스들의 순서도 고려 class D(C, B) 로 선언했다면, D 클래스의 \_\_mro\_\_ 도 변경된다. 

### Multiple Inheritance in the Real World

Mixin : 다중 상속을 통해 다른 클래스에 구상메서드를 제공하기 위해 설계된 클래스 

### Coping with Multiple Inheritance

1. Distinguish Interface Inheritance from Implementation Inheritance

- Inheritance of interface creates a subtype, implying an “is-a” relationship.
- Inheritance of implementation avoids code duplication by reuse.

코드 재사용을 위한 상속은 구현에 관련된 것

구성이나 위힘으로 대체 할 수도 있음

인터페이스 상속은 프레임워크의 중추적인 역할 

2. Make Interfaces Explicit with ABCs

ABC 로 명시하기 위해 abc.ABC 나 다른 ABC 를 상속하는 것

3. Use Mixins for Code Reuse

서로 관련없는 여러 서브클래스에서 코드를 재사용하기 위해 명시적으로 mixin 클래스로 만들어 함. 

mixin 클래스는 새로운 자료형을 정의하지 않고, 단지 재 사용할 메서드들을 묶어 놓음

mixin 클래스로 객체를 생성하면 안되며, mixin 상속하는 구상 클래스는 다른 클래스도 상속해야 함

각각의 mixin 클래스는 밀접히 연관된 메서드 몇개를 구현해서 하나의 구체적인 행위를 제공 해야함

4. Make Mixins Explicit by Naming

5. An ABC May Also Be a Mixin; The Reverse Is Not True

ABC 는 구상으로도 mixin 으로도 사용 가능 

ABC 는 자료형을 정의하지만, mixin 은 정의하지 않음

ABC 는 다른 클레스의 유일한 기본 클래스가 될 수 있지만, mixin 은 하나의 클래스로만 정의되선 안됨 

ABC 에 구현된 구상 메서드는 해당 ABC나 슈퍼클래스의 메서드만 사용 가능 

6. Don’t Subclass from More Than One Concrete Class

구상 클래스의 슈퍼클래스중 하나를 제외한 나머지 클래스는 ABC 나 mixin 이어야 함

7. Provide Aggregate Classes to Users

Grady Booch calls this **an aggregate class**.

```python
class Widget(BaseWidget, Pack, Place, Grid):
    """Internal class.
    
    Base class for a widget which can be positioned with the
    geometry managers Pack, Place or Grid."""
    pass
```

8. “Favor Object Composition Over Class Inheritance.”

단일 상속의 경우에도 구성 방식은 융통성을 향상시킴. 상속은 클래스들을 강하게 결합시키며, 상속 트리의 높이가 높아지면 구조가 불안정해짐

this principle enhances flexibility, because subclassing is a form of tight coupling, and tall inheritance trees tend to be brittle.

다른 클래스에 서비스를 제공하기 위해 사용할 때는 구성과 위임이 mixin 을 대체 할 수 있지만 자료형의 계층 구조를 정의 할 때는 구성과 위임이 interface 상속을 대처 할 수 없음 